The goal of this document is to scrape some ebay data to get GPU prices
I intend to use the beautiful soup package

In [1]:
from bs4 import BeautifulSoup
import ssl
import urllib.request, urllib.parse
import pandas as pd
import time

URLS

In [2]:
# 3060
url="https://www.ebay.com/sch/27386/i.html?_from=R40&_nkw=rtx+3060&LH_ItemCondition=1500%7C2000%7C2500%7C3000%7C1000&LH_Sold=1&LH_Complete=1&_ipg=240"

Request

In [3]:
html=urllib.request.urlopen(url).read()
soup=BeautifulSoup(html,'html.parser')

Get the information about each item

In [4]:
main_data=soup.find_all('div',class_="s-item__info clearfix")

Now parse this & add to df

In [5]:
names=[]
prices=[]
i=0
for line in main_data:
  #print(i)
  names.append(line.select_one(".s-item__title span").text)
  prices.append(line.find("span",class_="s-item__price").get_text())

  #df['price'][i]=price
prices = [price.replace("$", "") for price in prices]
prices = [price.replace(",", "") for price in prices]
prices = [float(price) for price in prices]
#prices=float(prices)
df = pd.DataFrame({
  "name": names,
  "price": prices
})
df=df.drop(0)

In [6]:
#print(df.head())

print('Max Price',df['price'].max())
print('Mean Price',df['price'].mean())
print('Min Price',df['price'].min())

Max Price 500.0
Mean Price 234.19925
Min Price 7.58


In [7]:
#df[df['price']>50]
print('Mean with values below 50 removed: ',df[df['price']>50]['price'].mean())
print('Max with values below 50 removed: ',df[df['price']>50]['price'].max())
print('Min with values below 50 removed: ',df[df['price']>50]['price'].min())

Mean with values below 50 removed:  237.0027848101266
Max with values below 50 removed:  500.0
Min with values below 50 removed:  56.0


In [8]:
# GPU's I want to search
GPUs=['RTX 4090', 'RTX 4080', 'RX 7900 XTX', 'RTX 4070 Ti', 'RTX 3090 Ti',
      'RX 6950 XT', 'RX 7900 XT', 'RTX 3080 Ti', 'RTX 3090', 'RTX 3080',
      'RX 6900', 'RX 6800', 'RTX 3070', 'RTX 3070 Ti', 'RTX 2080 Ti',
      'RX 6800 XT', 'RX 6750 XT', 'RTX 3060 Ti', 'RX 6700 XT', 'RTX 2080 super',
      'RTX 2080', 'GTX 1080 Ti', 'RTX 2070 Super', 'RX 6650 XT', 'RTX 3060',
      'RX 5700 XT', 'RTX 2060 Super', 'RX 6600 XT', 'RTX 2070', 'RTX 2060',
      'GTX 1080', 'RX 6600', 'RX 5700', 'GTX 1070 Ti', 'RTX 2060', 'GTX 980 Ti',
      'RX 5600 XT', 'GTX 1070', 'GTX 1660 Super', 'GTX 1660', 'GTX 3050',
      'GTX 1650 Super','GTX 1060','RX 6500 XT', 'GTX 760 Ti', 'GTX 750 Ti',
      'GTX 1050']
CPUs= ['Ryzen 5 3600', 'Ryzen 5 5500', 'Ryzen 5 5600', 'Ryzen 5 4600G',
       'Ryzen 7 2700X', 'Ryzen 5 3600X', 'Ryzen 7 4700', 'Ryzen 7 5700',
       'Ryzen 5 5600X', 'Ryzen 7 5800', 'Ryzen 5 7600X', 'Ryzen 5 7600',
       'Ryzen 9 5900', 'Ryzen 9 3950X']
Motherboards= ['b450', 'b550', 'b460', 'b560']

Things to do.
1. make a loop that goes through all the items in the GPUs list.
2. For each item in the loop, replace the space with +.
3. Do a search for those terms & make a similar dataframe to before
4. Put the name, min, max, mean, sd in a master dataframe.

In [9]:
master_df = pd.DataFrame(columns=['name', 'min price', 'mean price', 'max price'])
master_df['name']=GPUs
# what is the length of GPUs?
# print('Number of GPUs to search: ',len(GPUs))
# 'https://www.ebay.com/sch/27386/i.html?_from=R40&_nkw=RX%206950%20XT&LH_ItemCondition=1500%7C2000%7C2500%7C3000%7C1000&LH_Sold=1&LH_Complete=1&_ipg=240&rt=nc&_udlo=50'
# what is the string to put the search in?
urlp1="https://www.ebay.com/sch/27386/i.html?_from=R40&_nkw="
urlp2="&LH_ItemCondition=1500%7C2000%7C2500%7C3000%7C1000&LH_Sold=1&LH_Complete=1&_ipg=240&rt=nc&_udlo=50"
j=0
for gpu in GPUs:
  searchstr=(gpu.replace(" ", "%20"))
  url=str.join("", [urlp1,searchstr,urlp2])
  #print(url)
  html=urllib.request.urlopen(url).read()
  soup=BeautifulSoup(html,'html.parser')
  main_data=soup.find_all('div',class_="s-item__info clearfix")

  names=[]
  prices=[]
  i=0
  for line in main_data:
    #print(i)
    if "to" not in line.find("span",class_="s-item__price").get_text():
      #print("yes")
      names.append(line.select_one(".s-item__title span").text)
      prices.append(line.find("span",class_="s-item__price").get_text())

    #df['price'][i]=price
  prices = [price.replace("$", "") for price in prices]
  prices = [price.replace(",", "") for price in prices]
  prices = [float(price) for price in prices]
  #prices=float(prices)
  df = pd.DataFrame({
    "name": names,
    "price": prices
  })
  df=df.drop(0)
  #print('Max Price',df['price'].max())
  #print('Mean Price',df['price'].mean())
  #print('Min Price',df['price'].min())

#time.sleep(0.1)
  master_df['min price'][j]=df['price'].min()
  master_df['mean price'][j]=df['price'].mean()
  master_df['max price'][j]=df['price'].max()
  j=j+1
  #print(master_df.head())



In [ ]:
print(master_df)

              name min price   mean price max price
0         RTX 4090     64.99  1538.855708    3150.0
1         RTX 4080     59.99  1066.052208    1695.0
2      RX 7900 XTX    649.99   922.295189   1562.88
3      RTX 4070 Ti     100.0   809.112833    2299.0
4      RTX 3090 Ti     77.98   763.097333   2276.99
5       RX 6950 XT      86.0   619.373308     900.0
6       RX 7900 XT    649.99   783.459333   1129.99
7      RTX 3080 Ti     102.5   651.785941    1400.0
8         RTX 3090     77.98   763.097333   2276.99
9         RTX 3080     318.0   564.173708   1698.88
10         RX 6900     250.0   550.733583    1399.0
11         RX 6800     275.0   437.710708    866.66
12        RTX 3070     187.5   346.294375     700.0
13     RTX 3070 Ti     243.5   349.408042    962.83
14     RTX 2080 Ti     99.95   373.314603     998.0
15      RX 6800 XT     350.0      472.894    866.66
16      RX 6750 XT     281.0   347.799444    500.41
17     RTX 3060 Ti     102.5   305.161125     799.0
18      RX 6

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [ ]:
# Save the object to a file in your Google Drive
master_df.to_csv('/content/gdrive/My Drive/master_GPU_df.csv', index=False)


Now for CPUs

In [ ]:
cpu_master_df = pd.DataFrame(columns=['name', 'min price', 'mean price', 'max price'])
cpu_master_df['name']=CPUs
# 'https://www.ebay.com/sch/i.html?_from=R40&_nkw=ryzen+5+3600&_sacat=164&LH_Sold=1&LH_Complete=1&rt=nc&LH_ItemCondition=1000%7C1500%7C2500%7C3000&_ipg=240'
# what is the string to put the search in?
urlp1="https://www.ebay.com/sch/i.html?_from=R40&_nkw="
urlp2="&_sacat=164&LH_Sold=1&LH_Complete=1&rt=nc&LH_ItemCondition=1000%7C1500%7C2500%7C3000&_ipg=240"
j=0
for cpu in CPUs:
  searchstr=(cpu.replace(" ", "+"))
  url=str.join("", [urlp1,searchstr,urlp2])
  #print(url)
  html=urllib.request.urlopen(url).read()
  soup=BeautifulSoup(html,'html.parser')
  main_data=soup.find_all('div',class_="s-item__info clearfix")

  names=[]
  prices=[]
  i=0
  for line in main_data:
    #print(i)
    if "to" not in line.find("span",class_="s-item__price").get_text():
      #print("yes")
      names.append(line.select_one(".s-item__title span").text)
      prices.append(line.find("span",class_="s-item__price").get_text())

    #df['price'][i]=price
  prices = [price.replace("$", "") for price in prices]
  prices = [price.replace(",", "") for price in prices]
  prices = [float(price) for price in prices]
  #prices=float(prices)
  df = pd.DataFrame({
    "name": names,
    "price": prices
  })
  df=df.drop(0)
  #print('Max Price',df['price'].max())
  #print('Mean Price',df['price'].mean())
  #print('Min Price',df['price'].min())

#time.sleep(0.1)
  cpu_master_df['min price'][j]=df['price'].min()
  cpu_master_df['mean price'][j]=df['price'].mean()
  cpu_master_df['max price'][j]=df['price'].max()
  j=j+1
  #print(master_df.head())

print(cpu_master_df)

             name min price  mean price max price
0    Ryzen 5 3600     19.98    72.37521    399.95
1    Ryzen 5 5500       5.0   84.133233    179.99
2    Ryzen 5 5600      15.0   116.63654    196.14
3   Ryzen 5 4600G      22.0   71.690039     300.0
4   Ryzen 7 2700X     21.99    91.32135    319.99
5   Ryzen 5 3600X      16.1   90.423125    259.99
6    Ryzen 7 4700      35.0  132.221741     310.0
7    Ryzen 7 5700      60.0  165.384519     280.0
8   Ryzen 5 5600X      4.99  124.251176    205.52
9    Ryzen 7 5800     59.99  210.548383     460.0
10  Ryzen 5 7600X     135.0  201.841979     275.0
11   Ryzen 5 7600     135.0  200.705648     275.0
12   Ryzen 9 5900     170.1  281.392385    2399.0
13  Ryzen 9 3950X     175.0   327.48901     630.0


In [ ]:
cpu_master_df.to_csv('/content/gdrive/My Drive/master_CPU_df.csv', index=False)

Motherboards

M

In [ ]:
moth_master_df = pd.DataFrame(columns=['name', 'min price', 'mean price', 'max price'])
moth_master_df['name']= Motherboards
# 'https://www.ebay.com/sch/i.html?_from=R40&_nkw=b450%20&_sacat=1244&LH_TitleDesc=0&LH_Sold=1&LH_Complete=1&LH_ItemCondition=1000%7C1500%7C2500%7C3000&_ipg=240&rt=nc&_udlo=30'
# what is the string to put the search in?
urlp1="https://www.ebay.com/sch/i.html?_from=R40&_nkw="
urlp2="%20&_sacat=1244&LH_TitleDesc=0&LH_Sold=1&LH_Complete=1&LH_ItemCondition=1000%7C1500%7C2500%7C3000&_ipg=240&rt=nc&_udlo=30"
j=0
for moth in Motherboards:
  searchstr=(moth.replace(" ", "+"))
  url=str.join("", [urlp1,searchstr,urlp2])
  #print(url)
  html=urllib.request.urlopen(url).read()
  soup=BeautifulSoup(html,'html.parser')
  main_data=soup.find_all('div',class_="s-item__info clearfix")

  names=[]
  prices=[]
  i=0
  for line in main_data:
    #print(i)
    if "to" not in line.find("span",class_="s-item__price").get_text():
      #print("yes")
      names.append(line.select_one(".s-item__title span").text)
      prices.append(line.find("span",class_="s-item__price").get_text())

    #df['price'][i]=price
  prices = [price.replace("$", "") for price in prices]
  prices = [price.replace(",", "") for price in prices]
  prices = [float(price) for price in prices]
  #prices=float(prices)
  df = pd.DataFrame({
    "name": names,
    "price": prices
  })
  df=df.drop(0)
  #print('Max Price',df['price'].max())
  #print('Mean Price',df['price'].mean())
  #print('Min Price',df['price'].min())

#time.sleep(0.1)
  moth_master_df['min price'][j]=df['price'].min()
  moth_master_df['mean price'][j]=df['price'].mean()
  moth_master_df['max price'][j]=df['price'].max()
  j=j+1
  #print(master_df.head())

print(moth_master_df)

   name min price mean price max price
0  b450      35.0  84.288912     425.0
1  b550      35.0  112.40437    499.95
2  b460      30.0  89.473533   1203.75
3  b560      31.0  97.197458     240.9


In [ ]:
moth_master_df.to_csv('/content/gdrive/My Drive/master_motherboards_df.csv', index=False)